In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Arrhythmia"
cohort = "GSE143924"

# Input paths
in_trait_dir = "../../input/GEO/Arrhythmia"
in_cohort_dir = "../../input/GEO/Arrhythmia/GSE143924"

# Output paths
out_data_file = "../../output/preprocess/Arrhythmia/GSE143924.csv"
out_gene_data_file = "../../output/preprocess/Arrhythmia/gene_data/GSE143924.csv"
out_clinical_data_file = "../../output/preprocess/Arrhythmia/clinical_data/GSE143924.csv"
json_path = "../../output/preprocess/Arrhythmia/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Transcriptome analysis from human epicardial adipose tissue biopsies analyzed according to postoperative atrial fibrillation occurrence after cardiac surgery"
!Series_summary	"Introduction: Post-operative atrial fibrillation (POAF) is a frequent complication after cardiac surgery, but its pathophysiology remains incompletely understood. Considering that epicardial adipose tissue (EAT) is in close vicinity with the atrial myocardium, we hypothesized that a specific pre-operative EAT phenotype would be associated to POAF onset following surgery.  Methods: Patients undergoing cardiac surgery prospectively enrolled in the POMI-AF cohort between February 2016 and June 2017 were studied. EAT samples were collected at the beginning of surgery. Whole-tissue gene expression patterns and the stromal and vascular fraction (SVF) cellular composition were explored. Patients were followed after surgery by continuous ECG to detect POAF onset.  Results: Among the

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
import pandas as pd
import os
import json
from typing import Dict, Any, Optional, Callable

# 1. Gene Expression Data Availability
# Based on the series title and summary, this dataset appears to contain transcriptome analysis
# which typically includes gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# For trait (Arrhythmia/POAF), we can see it's available in row 1 of the sample characteristics
trait_row = 1

# For age and gender, they don't appear to be explicitly mentioned in the sample characteristics
age_row = None
gender_row = None

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert trait values to binary format (0 for sinus rhythm, 1 for POAF)"""
    if isinstance(value, str):
        if "sinus rhythm" in value.lower():
            return 0
        elif "postoperative atrial fibrillation" in value.lower() or "poaf" in value.lower():
            return 1
    return None

def convert_age(value):
    """Convert age values to continuous format"""
    # Since age data is not available, this function is just a placeholder
    return None

def convert_gender(value):
    """Convert gender values to binary format (0 for female, 1 for male)"""
    # Since gender data is not available, this function is just a placeholder
    return None

# 3. Save Metadata
# Determine trait data availability
is_trait_available = trait_row is not None

# Conduct initial filtering and save metadata
validate_and_save_cohort_info(
    is_final=False,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available
)

# 4. Clinical Feature Extraction
# Only proceed if trait_row is not None
if trait_row is not None:
    # Load the clinical data
    clinical_data_file = os.path.join(in_cohort_dir, "clinical_data.csv")
    if os.path.exists(clinical_data_file):
        clinical_data = pd.read_csv(clinical_data_file)
        
        # Extract clinical features
        selected_clinical_df = geo_select_clinical_features(
            clinical_df=clinical_data,
            trait=trait,
            trait_row=trait_row,
            convert_trait=convert_trait,
            age_row=age_row,
            convert_age=convert_age,
            gender_row=gender_row,
            convert_gender=convert_gender
        )
        
        # Preview the dataframe
        preview = preview_df(selected_clinical_df)
        print("Preview of selected clinical features:")
        print(preview)
        
        # Create directory if it doesn't exist
        os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
        
        # Save to CSV
        selected_clinical_df.to_csv(out_clinical_data_file, index=False)
        print(f"Clinical data saved to {out_clinical_data_file}")
    else:
        print(f"Clinical data file not found at {clinical_data_file}")


Clinical data file not found at ../../input/GEO/Arrhythmia/GSE143924/clinical_data.csv


### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Arrhythmia/GSE143924/GSE143924_series_matrix.txt.gz
Gene data shape: (8596, 30)
First 20 gene/probe identifiers:
Index(['A2M-AS1', 'AACS', 'AADAC', 'AADACL2', 'AADACP1', 'AAK1', 'AARD',
       'AASS', 'ABCA1', 'ABCA10', 'ABCA11P', 'ABCA3', 'ABCA5', 'ABCA9-AS1',
       'ABCB1', 'ABCB11', 'ABCB4', 'ABCB7', 'ABCC3', 'ABCC6'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These identifiers appear to be standard human gene symbols.
# Examples like A2M-AS1, AACS, AADAC, etc. are recognized gene symbols
# rather than probe identifiers that would need mapping.

requires_gene_mapping = False


### Step 5: Data Normalization and Linking

In [6]:
# 1. Normalize gene symbols in the gene expression data
# Use normalize_gene_symbols_in_index to standardize gene symbols
normalized_gene_data = normalize_gene_symbols_in_index(gene_data)
print(f"Gene data shape before normalization: {gene_data.shape}")
print(f"Gene data shape after normalization: {normalized_gene_data.shape}")

# Save the normalized gene data to file
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
normalized_gene_data.to_csv(out_gene_data_file)
print(f"Normalized gene expression data saved to {out_gene_data_file}")

# Load the actual clinical data from the matrix file that was previously obtained in Step 1
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
background_info, clinical_data = get_background_and_clinical_data(matrix_file)

# Get preview of clinical data to understand its structure
print("Original clinical data preview:")
print(clinical_data.head())

# 2. If we have trait data available, proceed with linking
if trait_row is not None:
    # Extract clinical features using the original clinical data
    selected_clinical_df = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )

    print(f"Selected clinical data shape: {selected_clinical_df.shape}")
    print("Clinical data preview:")
    print(selected_clinical_df.head())

    # Link the clinical and genetic data
    linked_data = geo_link_clinical_genetic_data(selected_clinical_df, normalized_gene_data)
    print(f"Linked data shape before processing: {linked_data.shape}")
    print("Linked data preview (first 5 rows, 5 columns):")
    print(linked_data.iloc[:5, :5] if not linked_data.empty else "Empty dataframe")

    # 3. Handle missing values
    try:
        linked_data = handle_missing_values(linked_data, trait)
        print(f"Data shape after handling missing values: {linked_data.shape}")
    except Exception as e:
        print(f"Error handling missing values: {e}")
        linked_data = pd.DataFrame()  # Create empty dataframe if error occurs

    # 4. Check for bias in features
    if not linked_data.empty and linked_data.shape[0] > 0:
        is_biased, linked_data = judge_and_remove_biased_features(linked_data, trait)
        print(f"Data shape after removing biased features: {linked_data.shape}")
    else:
        is_biased = True
        print("Cannot check for bias as dataframe is empty or has no rows after missing value handling")

    # 5. Validate and save cohort information
    note = ""
    if linked_data.empty or linked_data.shape[0] == 0:
        note = "Dataset contains gene expression data related to atrial fibrillation after cardiac surgery, but linking clinical and genetic data failed, possibly due to mismatched sample IDs."
    else:
        note = "Dataset contains gene expression data for atrial fibrillation after cardiac surgery, which is relevant to arrhythmia research."
    
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=True,
        is_biased=is_biased,
        df=linked_data,
        note=note
    )

    # 6. Save the linked data if usable
    if is_usable:
        os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
        linked_data.to_csv(out_data_file)
        print(f"Linked data saved to {out_data_file}")
    else:
        print("Dataset is not usable for analysis. No linked data file saved.")
else:
    # If no trait data available, validate with trait_available=False
    is_usable = validate_and_save_cohort_info(
        is_final=True,
        cohort=cohort,
        info_path=json_path,
        is_gene_available=True,
        is_trait_available=False,
        is_biased=True,  # Set to True since we can't use data without trait
        df=pd.DataFrame(),  # Empty DataFrame
        note="Dataset contains gene expression data but lacks proper clinical trait information for arrhythmia analysis."
    )
    
    print("Dataset is not usable for arrhythmia analysis due to lack of clinical trait data. No linked data file saved.")

Gene data shape before normalization: (8596, 30)
Gene data shape after normalization: (7873, 30)


Normalized gene expression data saved to ../../output/preprocess/Arrhythmia/gene_data/GSE143924.csv


Original clinical data preview:
         !Sample_geo_accession                                     GSM4276706  \
0  !Sample_characteristics_ch1              tissue: epicardial adipose tissue   
1  !Sample_characteristics_ch1  patient diagnosis: sinus rhythm after surgery   

                                      GSM4276707  \
0              tissue: epicardial adipose tissue   
1  patient diagnosis: sinus rhythm after surgery   

                                      GSM4276708  \
0              tissue: epicardial adipose tissue   
1  patient diagnosis: sinus rhythm after surgery   

                                      GSM4276709  \
0              tissue: epicardial adipose tissue   
1  patient diagnosis: sinus rhythm after surgery   

                                      GSM4276710  \
0              tissue: epicardial adipose tissue   
1  patient diagnosis: sinus rhythm after surgery   

                                      GSM4276711  \
0              tissue: epicardial adipose ti

Data shape after handling missing values: (30, 7874)
For the feature 'Arrhythmia', the least common label is '0.0' with 15 occurrences. This represents 50.00% of the dataset.
The distribution of the feature 'Arrhythmia' in this dataset is fine.

Data shape after removing biased features: (30, 7874)


Linked data saved to ../../output/preprocess/Arrhythmia/GSE143924.csv
